# Logfiles analysis

Let us inspect the result of a BigDFT calculation from the `Logfile` class.

An instance is returned after the execution is finished in the form of a `yaml` file, from which the `Logfile` object is extracted.

Let us for example investigate the logfile of the LDA calculation of N2.

In [ ]:
install = "client (Google drive)" #@param ["full_suite", "client (Google drive)", "client"]
install_var=install
!wget https://raw.githubusercontent.com/BigDFT-group/bigdft-school/data/packaging/install.py &> /dev/null
args={'locally': True} if install == 'client' else {}
import install
#getattr(install,install_var.split()[0])(**args)

In [ ]:
!pip install pybigdft py3dmol spglib ase -q


In [ ]:
install.data('data/session1.tar.xz')

In [ ]:
from BigDFT import Logfiles as L

logfile = L.Logfile('./scratch/log-xc__LDA.yaml')

From this instance, it is first possible to visualize the associated system

In [ ]:
from BigDFT import Systems as S

sys = S.system_from_log(logfile,fragmentation='full')
sys.display();

The information of the logfile are presented also if the class instance is printed

---



In [ ]:
print(logfile)

The `yaml` serialization is easily obtainable by calling the `log` attribute, in the form of a `dict`. However, this serialization is not particularly useful to visualize the output. Better to refer directly to the logfile.

In [ ]:
from yaml import dump
print(dump(logfile.log))

For example, information on the Poisson solver is accessible by

In [ ]:
logfile.log["Poisson Solver"]

Or similarly, the self-consistent field cycle

In [ ]:
logfile.SCF_convergence()

All `keys` elements are listed afterwards

In [ ]:
print(logfile.log.keys())

## Extended systems: graphene

Let us inspect a simple example of a solid state calculation by considering a two-dimensional (2D) materials, graphene.

In [ ]:
gr = L.Logfile("./scratch/log-graphene.yaml")

[Graphene](https://en.wikipedia.org/wiki/Graphene) is a 2D carbon allotrope in the form of a honeycomb network that consists of a two inequivalent triangular lattices, with a C-C bond of 1.42$~$angstroem, or equivalently, a lattice parameter such that $a_0 = 2.46~$angstroem.

In [ ]:
# or gr.log["Atomic structure"]
gr.astruct

In [ ]:
# this only visualizes the unit cell atoms
sys = S.system_from_log(gr,fragmentation='full')
sys.display();

In order to properly characterize extended systems, a $k$-points mesh is required to handle periodicity.

In [ ]:
gr.kpt_mesh

Additionally, mixing is sometimes needed for convergence, such that the effect of abrupt changes in the density are smoothened away.

In [ ]:
print(dump(gr.log["mix"]))
# inp["import"] = "mixing"

The interested reader is invited to go through this [tutorial](https://l_sim.gitlab.io/bigdft-suite/tutorials/SolidState.html)

### Graphene massless Dirac fermions (optional)

When analyzing electronic properties, the $k$-dependence is obtained by plotting the band structure. This is done along a given path, that can be specified by the user.

In [ ]:
BZ_gr = gr.get_brillouin_zone()

hsp = BZ_gr.special_points # high symmetry points
[print(i) for i in hsp.items()]
paths = BZ_gr.special_paths # high symmetry path list
print(paths)

Here we choose half the first path, since the 2D character of our system implies no dependence in $k_x$.

In [ ]:
path = paths[0][:5]

Eventually, the band structure is plotted using the `BZ` class by giving a list of special $k$-points (i.e. the `path` defined above)

In [ ]:
import BigDFT.BZ as BZ

npts = 200
path_bs = BZ.BZPath(BZ_gr.lattice,path,hsp,npts=npts)

ax = BZ_gr.plot(path=path_bs,npts=npts)
ax.set_ylim([-10,5]);